In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
dataset=pd.read_csv('../input/loan.csv', low_memory=False)

In [ ]:
dataset.head()

Next up, we want to get rid of sparsely populated variables, so we remove columns with more that 75% missing values

In [ ]:
var_null_pc=dataset.isnull().sum(axis=0).sort_values(ascending=False)/float(len(dataset))
var_null_pc[var_null_pc>0.75]

We have 20 variables which we can remove from modelling and analysis
We should also remove those observations where all values are NAs or nulls

In [ ]:
dataset.drop( var_null_pc[ var_null_pc > 0.75 ].index, axis = 1, inplace = True ) 
dataset.dropna( axis = 0, thresh = 30, inplace = True )

After having a closer look at the data (and studying frequency tables), we can remove the following variables, reasons described:

policy_code is always == 1
id and member_id are all unique, which is a bit misleading. So every record is a unique customer
application_type is 'INDIVIDUAL' for 99.94% of the records
zip_code not needed for this level of analysis,
title can be removed as well.

In [ ]:
vars_to_be_removed=['id','member_id','policy_code','zip_code','application_type']

dataset.drop( vars_to_be_removed , axis = 1, inplace = True )

In [ ]:
dataset.head()

In [ ]:
dataset.select_dtypes(include=[np.object]).isnull().sum()

treating missing value with datatype object using mode 

In [ ]:
for value in ('emp_title','emp_length','title','last_pymnt_d','next_pymnt_d','last_credit_pull_d','earliest_cr_line'):
    dataset[value].fillna(dataset[value].mode()[0],inplace=True)


In [ ]:
dataset.select_dtypes(include=[np.object]).isnull().sum()

In [ ]:
dataset.select_dtypes(include=[np.number]).isnull().sum()

treating missing value with datatype int64, float64 using mean 

In [ ]:
import math
for value in ('annual_inc','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','open_acc','pub_rec','revol_util','total_acc',
             'collections_12_mths_ex_med','acc_now_delinq','tot_coll_amt','tot_cur_bal','total_rev_hi_lim'):
   dataset[value].fillna(math.floor(dataset[value].mean()),inplace=True) 

In [ ]:
dataset.select_dtypes(include=[np.number]).isnull().sum()

dropping url as it does not going to contribute for further prediction

In [ ]:
dataset.drop(['url'],axis=1,inplace=True)

In [ ]:
dataset['loan_status'].value_counts()

removing loan_status values referring to Issued,Does not meet the credit policy. Status:Fully Paid ,Does not meet the credit policy. Status:Charged Off

In [ ]:
dataset = dataset[~dataset['loan_status'].isin(['Issued',
                                 'Does not meet the credit policy. Status:Fully Paid',
                                 'Does not meet the credit policy. Status:Charged Off'
                                ])]

converting the loan_status column to 0 or 1 so to predict the loan defaulter where 0 is non Default while 1 is  default

In [ ]:
def CreateDefault(loan_status):
    if loan_status in ['Current', 'Fully Paid', 'In Grace Period']:
        return 0
    else:
        return 1 
dataset['Default'] = dataset['loan_status'].apply(lambda x: CreateDefault(x))

In [ ]:
dataset['Default']

Extract numbers from emp_length and fill missing values with the median. If emp_length == 10+ years then leave it as 10

In [ ]:
dataset['emp_length'] = dataset['emp_length'].str.replace('+','')

In [ ]:
dataset['emp_length'] = dataset['emp_length'].str.replace('<','')

In [ ]:
dataset['emp_length'].value_counts()

converting date format(year-month) to year format

In [ ]:
dataset['last_pymnt_d'] = pd.to_numeric(dataset['last_pymnt_d'].str[-4:], errors='coerce').round(0)

In [ ]:
dataset['issue_d'] = pd.to_numeric(dataset['issue_d'].str[-4:], errors='coerce').round(0)

In [ ]:
dataset['last_credit_pull_d'] = pd.to_numeric(dataset['last_credit_pull_d'].str[-4:], errors='coerce').round(0)

In [ ]:
dataset.drop(['next_pymnt_d'],axis=1,inplace=True)

In [ ]:
dataset.head()

In [ ]:
objects=dataset['emp_length'].unique()
print(objects)
import numpy as np
y_pos = np.arange(len(objects))

new_df=dataset.groupby("emp_length").sum()
#print(new_df)
total_attacks=new_df["Default"]
plt.bar(y_pos, total_attacks, alpha=1.0)
#plt.barh(y_pos, total_attacks)
plt.xticks(y_pos, objects,rotation=90)
plt.xlabel("emp_length")
plt.ylabel('Default')
plt.title('Defaulters vs emp_length')
plt.show()

In [ ]:
objects=dataset['term'].unique()
print(objects)
import numpy as np
y_pos = np.arange(len(objects))

new_df=dataset.groupby("term").sum()
#print(new_df)
total_attacks=new_df["Default"]
plt.bar(y_pos, total_attacks, alpha=1.0)
#plt.barh(y_pos, total_attacks)
plt.xticks(y_pos, objects,rotation=360)
plt.xlabel("term")
plt.ylabel('Default')
plt.title('Defaulters vs term')
plt.show()

In [ ]:
dataset['home_ownership'].unique()

In [ ]:
dataset['home_ownership'].value_counts()

In [ ]:
objects=dataset['home_ownership'].unique()
print(objects)
import numpy as np
y_pos = np.arange(len(objects))

new_df=dataset.groupby("home_ownership").sum()
#print(new_df)
total_attacks=new_df["Default"]
plt.bar(y_pos, total_attacks, alpha=1.0)
#plt.barh(y_pos, total_attacks)
plt.xticks(y_pos, objects,rotation=360)
plt.xlabel("home_ownership")
plt.ylabel('Default')
plt.title('Defaulters vs home_ownership')
plt.show()

In [ ]:
dataset.dtypes

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 3))

sns.distplot(dataset[dataset['Default'] == 0]['int_rate'], bins=30, ax=ax1, kde=False)
sns.distplot(dataset[dataset['Default'] == 1]['int_rate'], bins=30, ax=ax2, kde=False)

ax1.set_title('No Default')
ax2.set_title('Default')

ax1.set_xbound(lower=0)
ax2.set_xbound(lower=0)

plt.tight_layout()
plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 3))

sns.distplot(dataset[dataset['Default'] == 0]['loan_amnt'], bins=40, ax=ax1, kde=False)
sns.distplot(dataset[dataset['Default'] == 1]['loan_amnt'], bins=40, ax=ax2, kde=False)

ax1.set_title('No Default')
ax2.set_title('Default')

ax1.set_xbound(lower=0)
ax2.set_xbound(lower=0)

plt.tight_layout()
plt.show()

In [ ]:
colname = ['term', 'grade','sub_grade', 'home_ownership','emp_length','emp_title', 'verification_status','loan_status','pymnt_plan',
           'purpose','addr_state','earliest_cr_line','title','initial_list_status']
colname

In [ ]:
from sklearn import preprocessing
le={}
#iterate through columns and assigning it with labels 
for x in colname:
    le[x]=preprocessing.LabelEncoder()
# iterate through colname and comparing it with the labels assigned to them in le and repalcing it with label value    
for x in colname:
        dataset[x]=le[x].fit_transform(dataset[x])

In [ ]:
dataset.dtypes

In [ ]:
loan_correlation = dataset.corr()
loan_correlation

In [ ]:
f, ax = plt.subplots(figsize=(14, 9))
sns.heatmap(loan_correlation, 
            xticklabels=loan_correlation.columns.values,
            yticklabels=loan_correlation.columns.values,annot= True)
plt.show()

In [ ]:
vars_to_be_removed=['loan_amnt','funded_amnt','funded_amnt_inv','verification_status','sub_grade','loan_status','total_pymnt','total_pymnt_inv','installment','issue_d','out_prncp_inv','out_prncp',
                    'total_rec_prncp','revol_bal','total_rec_int']
dataset.drop(vars_to_be_removed,axis=1,inplace=True)

In [ ]:
# create X and Y
Y=dataset.values[:,-1]
# it will store in the form of array and we pass -1 to subset an aaray , here  only last column
X=dataset.values[:,:-1]
# all rowsand and columns except last column

In [ ]:
Y=Y.astype(int)
print(Y)

In [ ]:
# splitting the data into train and set => train_test_split()
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)

In [ ]:
#building,training and testing the model

from sklearn.linear_model import LogisticRegression
classifier=(LogisticRegression())
#fitting the data to model
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)
print("Classification report:")

print(classification_report(Y_test, Y_pred))

acc=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model:",acc)

In [ ]:
#predicting using the Decision_Tree_Classifier
from sklearn.tree import DecisionTreeClassifier

model_DecisionTree = DecisionTreeClassifier()
model_DecisionTree.fit(X_train,Y_train)

In [ ]:
#fit the model on the data and predict the values
Y_pred = model_DecisionTree.predict(X_test)
#print(Y_pred)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
#confusion matrix
print(confusion_matrix(Y_test,Y_pred))
print(accuracy_score(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))